# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
#if secrets.py is in the same directory as this file, there was an error


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks=pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import IEX Cloud API token.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from api import IEX_CLOUD_API_TOKEN

## Making First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'avgTotalVolume': 58606100,
 'calculationPrice': 'close',
 'change': 0.2997,
 'changePercent': 0.00169,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 177.87,
 'iexCloseTime': 1691783999809,
 'iexLastUpdated': 1691783999809,
 'iexMarketPercent': 5.696896332447155,
 'iexOpen': 177.31,
 'iexOpenTime': 1691760600117,
 'iexRealtimePrice': 177.87,
 'iexRealtimeSize': 100,
 'iexVolume': 743576,
 'lastTradeTime': 1691783999997,
 'latestPrice': 177.79,
 'latestSource': 'Close',
 'latestTime': 'August 11, 2023',
 'latestUpdate': 1691784000407,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 27

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data["latestPrice"]
market_cap = data["marketCap"]

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns  = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
aapl = [symbol, price, market_cap, "N/A"]
type(aapl)
final_dataframe.loc[len(final_dataframe)] = aapl
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,177.79,2779610107280,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
import json
"""
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"]:
    api_url = f"https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = json.loads(response.content)
        final_dataframe.loc[len(final_dataframe)] = [stock, data["latestPrice"], data["marketCap"], "N/A"]
    else:
        print(f"Error getting data for {stock}: {response.status_code}")

"""

'\nfinal_dataframe = pd.DataFrame(columns = my_columns)\nfor stock in stocks["Ticker"]:\n    api_url = f"https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"\n    response = requests.get(api_url)\n\n    if response.status_code == 200:\n        data = json.loads(response.content)\n        final_dataframe.loc[len(final_dataframe)] = [stock, data["latestPrice"], data["marketCap"], "N/A"]\n    else:\n        print(f"Error getting data for {stock}: {response.status_code}")\n\n'

In [9]:

stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

In [10]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,177.79,2779610107280,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f"https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    #print(data.status_code)
    for symbol in symbol_string.split(","):
        final_dataframe.loc[len(final_dataframe)] = [symbol, data[symbol]["quote"]["latestPrice"], data[symbol]["quote"]["marketCap"], "N/A"]
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.86,37471376271,N/A
1,AAL,15.65,10225122123,N/A
2,AAP,72.28,4296601623,N/A
3,AAPL,177.79,2779610107280,N/A
4,ABBV,152.18,268604803762,N/A
...,...,...,...,...
496,YUM,134.20,37604353910,N/A
497,ZBH,123.93,25896925127,N/A
498,ZBRA,261.94,13447571066,N/A
499,ZION,38.30,5673958594,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [19]:
portfolio_size = input("Enter the value of your portfolio: ")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("Please enter an integer")

10000000.0


In [40]:
position_size = val/len(final_dataframe.index)
#print(position_size)
"""
number_of_apple_shares = position_size/final_dataframe[final_dataframe["Ticker"] == "AAPL"]["Stock Price"]
print(math.floor(number_of_apple_shares))
"""
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,126.86,37471376271,157
1,AAL,15.65,10225122123,1275
2,AAP,72.28,4296601623,276
3,AAPL,177.79,2779610107280,112
4,ABBV,152.18,268604803762,131
...,...,...,...,...
496,YUM,134.20,37604353910,148
497,ZBH,123.93,25896925127,161
498,ZBRA,261.94,13447571066,76
499,ZION,38.30,5673958594,521


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [62]:
writer = pd.ExcelWriter("recommended_trades_eq_weights.xlsx", engine="xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [63]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)
dollar_format = writer.book.add_format(

    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)
integer_format = writer.book.add_format(

    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }

)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [64]:
"""
writer.sheets["Recommended Trades"].set_column("A:A", 18, string_format)
writer.sheets["Recommended Trades"].set_column("B:B", 18, string_format)
writer.sheets["Recommended Trades"].set_column("C:C", 18, string_format)
writer.sheets["Recommended Trades"].set_column("D:D", 18, string_format)
writer.close()
"""

"""
writer.sheets["Recommended Trades"].write("A1","Ticker", string_format)
.
.
.
"""

'\nwriter.sheets["Recommended Trades"].write("A1","Ticker", string_format)\n.\n.\n.\n'

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Simplify this by putting it in 2 loops:

In [66]:
column_formats = {
    "A": ["Ticker", string_format],
    "B": ["Stock Price", dollar_format],
    "C": ["Market Capitalization", dollar_format],
    "D": ["Number of Shares to Buy", integer_format],
}

for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f"{column}:{column}", 18, column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close()